# import library

In [1]:
import nltk
import numpy as np
import pandas as pd
from nltk import tokenize
from nltk.stem import PorterStemmer 
from nltk.tag import StanfordPOSTagger
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
from nltk.corpus import sentiwordnet as swn
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.5/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


# module

In [43]:
def search_adj(sentence, keyword, skip_before_keyword=False, skip_after_keyword=False, prev_bag_of_word=1, after_bag_of_word=5) :    
    st = StanfordPOSTagger('./english-bidirectional-distsim.tagger', './stanford-postagger-3.9.2.jar')
    
    punctuation = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'] # remove it if you need punctuation 
    
    word_tokens = word_tokenize(sentence) 
    filtered_sentence = [w for w in word_tokens if not w in punctuation] 
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in punctuation:
            filtered_sentence.append(w)
            
    list_word_tag = st.tag([(" ").join(filtered_sentence)])
    found_keyword = not skip_before_keyword
    list_sentiment_word = []
    found_adjective = 0
    keyword_idx = -1
    for idx in range (len(list_word_tag)) :
        if (list_word_tag[idx][0] == keyword) :
            keyword_idx = idx
            break
    
    if (skip_before_keyword) :
        prev_bag_of_word = 0
    
    if (skip_after_keyword) :
        after_bag_of_word = 0
        
    for idx in range (len(list_word_tag)) :
        if (idx > keyword_idx + after_bag_of_word) :
                break
                
        if ((idx >= keyword_idx - prev_bag_of_word) and list_word_tag[idx][1] == 'JJ') :
            found_adjective += 1
            sentiment_word = list_word_tag[idx][0]
            for adverb_idx in range (idx-1, -1, -1) :
                if (list_word_tag[adverb_idx][1] == 'RB') :
                    sentiment_word = list_word_tag[adverb_idx][0] + " " + sentiment_word
                else :
                    break
            list_sentiment_word.append(sentiment_word)
            
    return (list_sentiment_word, found_adjective)

In [56]:
def preprocessing(document, skip_before_keyword=False, skip_after_keyword=False, prev_bag_of_word=1, after_bag_of_word=5) :
    act = ['acting','role playing','act',' actress','actor','role','portray','character','villain','performance', 'play', 'perform', 'doing']
    plot = ['plot','story','storyline','tale','romance','dialog','script','storyteller',' ending','storytelling','revenge','betrayal','writing']
    
    sid = SentimentIntensityAnalyzer()
    
    act_sentence = []
    plot_sentence = []
    act_score = 0
    plot_score = 0
    tokenizer = tokenize.PunktSentenceTokenizer()
    ps = PorterStemmer()
    
    list_sentence = tokenizer.sentences_from_text(document)
    for sentence in list_sentence :
        is_sentence_act = False
        is_sentence_plot = False
        list_word = tokenize.word_tokenize(sentence)
        for word in list_word :
            if (ps.stem(word.lower()) in act or word.lower() in act):
                list_adj_sentence, adj_found = search_adj(sentence, word, prev_bag_of_word=prev_bag_of_word, after_bag_of_word=after_bag_of_word)
                for adj_word in (list_adj_sentence) :
                    display(adj_word)
                    act_sentence.append(adj_word)
                    act_score += sid.polarity_scores(adj_word)['pos'] - sid.polarity_scores(adj_word)['neg']
            if (ps.stem(word.lower()) in plot or word.lower() in plot):
                list_adj_sentence, adj_found = search_adj(sentence, word, prev_bag_of_word=prev_bag_of_word, after_bag_of_word=after_bag_of_word)
                for adj_word in (list_adj_sentence) :
                    display(adj_word)
                    plot_sentence.append(adj_word)
                    plot_score +=  sid.polarity_scores(adj_word)['pos'] - sid.polarity_scores(adj_word)['neg']
#     return ([act_sentence,plot_sentence])
    return ([act_score,plot_score])

In [4]:
def list_to_sentence(list_of_word) :
    return((" ").join(list_of_word))

# CONTOH PENGGUNAAN

In [5]:
import time
start = time.time()

In [6]:
# input data
data = pd.read_excel("../input/datasetnlp_nographic_no_neutral.xlsx")
data = data
data.drop(columns=['title'], inplace=True)
for column_name in ['acting', 'plot'] :
    data.loc[data[column_name] == 'positive', column_name] = 1
    data.loc[data[column_name] == 'negative', column_name] = -1

In [7]:
# preprocessing
tokenizer = tokenize.PunktSentenceTokenizer()
data.text = data.text.apply(lambda x: preprocessing(x, prev_bag_of_word=1, after_bag_of_word=10))

In [8]:
data_act = pd.DataFrame([])
# data_act['text'] = [i[0] for i in data.iloc[:,0].tolist()]
# data_act.text = data_act.text.apply(lambda x: list_to_sentence(x))
data_act['sentiment_score'] = [i[0] for i in data.iloc[:,0].tolist()]
data_act['label'] = data.iloc[:,1]

In [9]:
data_plot = pd.DataFrame([])
# data_plot['text'] = [i[1] for i in data.iloc[:,0].tolist()]
# data_plot.text = data_plot.text.apply(lambda x: list_to_sentence(x))
data_plot['sentiment_score'] = [i[1] for i in data.iloc[:,0].tolist()]
data_plot['label'] = data.iloc[:,2]

In [10]:
# feature engineering
# from sklearn.feature_extraction.text import CountVectorizer
# count_vect = CountVectorizer()
# X_act_counts = count_vect.fit_transform(data_act.text)
# X_act_counts.shape

In [11]:
# X_plot_counts = count_vect.fit_transform(data_plot.text)
# X_plot_counts.shape

In [12]:
# from sklearn.feature_extraction.text import TfidfTransformer
# tfidf_transformer = TfidfTransformer()
# X_act_tfidf = tfidf_transformer.fit_transform(X_act_counts)
# X_act_tfidf.shape

In [13]:
# X_plot_tfidf = tfidf_transformer.fit_transform(X_plot_counts)
# X_plot_tfidf.shape

In [14]:
# feature_act = X_act_tfidf
# feature_plot = X_plot_tfidf
feature_act = pd.DataFrame(data_act['sentiment_score'])
feature_plot = pd.DataFrame(data_plot['sentiment_score'])

In [29]:
# # learning
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import confusion_matrix
# clf1 = MultinomialNB().fit(feature_act, data_act.label.tolist())
# clf2 = MultinomialNB().fit(feature_plot, data_plot.label.tolist())

In [ ]:
# naiv1 = clf1.predict(X_act_tfidf)
# naiv2 = clf2.predict(X_plot_tfidf)

In [16]:
# pred_svm1 = naiv1.predict(pd.DataFrame(feature_act))
# pred_svm2 = naiv2.predict(pd.DataFrame(feature_plot))

In [17]:
from sklearn import svm
svm1 = svm.SVC().fit(feature_act, data_act.label.tolist())
svm2 = svm.SVC().fit(feature_plot, data_plot.label.tolist())

/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [18]:
pred_svm1 = svm1.predict(pd.DataFrame(feature_act))
pred_svm2 = svm2.predict(pd.DataFrame(feature_plot))

In [19]:
display(f1_score(pred_svm1, data_act.label.tolist()),
        f1_score(pred_svm2, data_plot.label.tolist())
       )

0.967741935483871

0.8235294117647058

In [20]:
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier(n_estimators=10)
rf1.fit(feature_act, data_act.label.tolist())
rf2 = RandomForestClassifier(n_estimators=10)
rf2.fit(feature_plot, data_plot.label.tolist())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
pred_rf1 = rf1.predict(feature_act)
pred_rf2 = rf2.predict(feature_plot)
cv10_pred_rf1 = cross_val_predict(rf1, feature_act, data_act.label.tolist(), cv=10)
cv10_pred_rf2 = cross_val_predict(rf2, feature_plot, data_plot.label.tolist(), cv=10)

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [22]:
rf1.score(feature_act, data_act.label.tolist())

0.9375

In [23]:
rf2.score(feature_plot, data_plot.label.tolist())

0.71875

In [24]:
display(f1_score(cv10_pred_rf1, data_act.label.tolist()),
        f1_score(cv10_pred_rf2, data_plot.label.tolist())
       )

0.967741935483871

0.7878787878787878

In [25]:

from xgboost import XGBClassifier

xgb1 = XGBClassifier(max_depth=5,objective='binary:logistic',learning_rate=0.001,min_child_weight=1,scale_pos_weight=1)
xgb2 = XGBClassifier(max_depth=5,objective='binary:logistic',learning_rate=0.001,min_child_weight=1,scale_pos_weight=1)

predict_xgb_1 = cross_val_predict(xgb1, feature_act, data_act.label.tolist(), cv=10)
predict_xgb_2 = cross_val_predict(xgb2, feature_plot, data_plot.label.tolist(), cv=10)

cv10_predict_xgb_1 = xgb1.fit(feature_act, data_act.label.tolist()).predict(feature_act)
cv10_predict_xgb_2 = xgb2.fit(feature_plot, data_plot.label.tolist()).predict(feature_plot)

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [26]:
display(f1_score(predict_xgb_1, data_act.label.tolist()),
        f1_score(predict_xgb_2, data_plot.label.tolist()),
        f1_score(cv10_predict_xgb_1, data_act.label.tolist()),
        f1_score(cv10_predict_xgb_2, data_plot.label.tolist())
       )

0.967741935483871

0.7924528301886793

0.967741935483871

0.8235294117647058

In [85]:
def predict_sentiment(teks) :
    sentiment_score = preprocessing(teks)
    sentiment_score_act = pd.DataFrame([sentiment_score[0]], columns=["sentiment_score"])
    sentiment_score_plot = pd.DataFrame([sentiment_score[1]], columns=["sentiment_score"])
    act_sentiment = xgb1.predict(sentiment_score_act)
    plot_sentiment = xgb2.predict(sentiment_score_plot)
    
    return(act_sentiment[0], plot_sentiment[0])
    
    
act_sentiment, plot_sentiment = predict_sentiment('bad act')
display(act_sentiment, plot_sentiment)
if (act_sentiment == 1) :
    print("acting => positive")
else :
    print("acting => negative")
if (plot_sentiment == 1) :
    print("plot => positive")
else :
    print("plot => negative")

'bad'

1

1

acting => positive
plot => positive


In [101]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
new_dataset = smote.fit_resample

In [27]:
print (time.time() - start)

521.3793420791626


In [28]:
from nltk.corpus import sentiwordnet as swn

a = list(swn.senti_synsets('not'))
pos = 0
neg = 0
pos=pos+a[0].pos_score()
neg=neg+a[0].neg_score()
display(neg)

0.625